# Read and load results

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pypsa
from functions import *
np.set_printoptions(suppress=True)

# Define the use technologies and regions(s)
region = 'ESP'  
setup = {
    f'{region}': {
        'OCGT': True,
        'CCGT': False,
        'battery storage': True,
        'onwind': True,
        'offwind': False,
        'solar': True,
        'electrolysis': True,
        'fuel cell': True,
        'Hydrogen storage': True,
        'Reservoir hydro storage': True,
        'Load shedding': False
    }
}

# Default weather, hydro and demand years
w_year = 2011
h_year = 2007
d_year = 2018

# Dispatch and rolling horizon settings
w_year_dispatch = 2017
h_year_dispatch = 2007
d_year_dispatch = 2019

weather_years = All_data['solar'].index.year.unique()


In [3]:
d_year_selected = d_year_dispatch

base_dir = os.getcwd()  # Gets the current working directory (where notebook is running)
results_path_rh = os.path.join(base_dir, f"Network_results\\N_RH_d_{d_year_selected}")
results_path_pf = os.path.join(base_dir, f"Network_results\\N_PF_d_{d_year_selected}")
os.makedirs(results_path_rh, exist_ok=True)
os.makedirs(results_path_pf, exist_ok=True)

# NOGET HER VIRKER IKKE :O 
# Only if not recently saved networks 
file_names_rh = os.listdir(results_path_rh)
Saved_Networks_names_RHL = file_names_rh

file_names_pf = os.listdir(results_path_pf)
Saved_Networks_names_PF = file_names_pf


optmized_network_names_rh = []
optmized_network_names_pf = []

networks_pf = {}
networks_rh = {}


for i, n in enumerate(Saved_Networks_names_RHL):
    net_name = f'N_RHL_{weather_years[i]}_{region}'
    optmized_network_names_rh.append(net_name)
    path = os.path.join(results_path_rh, n)

    networks_rh[net_name] = pypsa.Network(path)
    #print(f"Loaded network: {net_name} from {n}")

for i, n in enumerate(Saved_Networks_names_PF):
    net_name = f'N_PF_{weather_years[i]}_{region}'
    optmized_network_names_pf.append(net_name)
    path = os.path.join(results_path_pf, n)

    networks_pf[net_name] = pypsa.Network(path)
    #print(f"Loaded network: {net_name} from {n}")
if len(networks_pf) != len(networks_rh) or len(networks_pf) != len(weather_years):
    print("Warning: The number of loaded PF and RH networks do not match!")

INFO:pypsa.io:Imported network N_RHL_w-1979_d-2019_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_RHL_w-1980_d-2019_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_RHL_w-1981_d-2019_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_RHL_w-1982_d-2019_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_RHL_w-1983_d-2019_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_RHL_w-1984_d-2019_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_RHL_w-1985_d-2019_h-2007_ESP.nc has buses, carriers, generators, links, loads, storage_units, stores
INFO:pypsa.io:Imported network N_RHL_w-1986_d-2019_h-2007_ESP.

In [7]:
optmized_network_names_rh[33]

'N_RHL_2012_ESP'

# Build df and tables to store results

In [10]:
networks_rh[optmized_network_names_rh[10]].buses_t['marginal_price']["electricity bus"].values

array([65.4237284, 65.4237284, 65.4237284, ..., 65.4237284, 65.4237284,
       65.4237284], shape=(8760,))

In [12]:
# Results storage
results = []

for i, (name_pf, name_rh) in enumerate(zip(optmized_network_names_pf, optmized_network_names_rh)):
    pf_net = networks_pf[name_pf]
    rh_net = networks_rh[name_rh]
    print(i)

    pf_cost = (pf_net.buses_t['marginal_price']["electricity bus"].values * pf_net.loads_t['p_set']['load']).sum() / 1e6
    rh_cost = (rh_net.buses_t['marginal_price']["electricity bus"].values * rh_net.loads_t['p_set']['load']).sum() / 1e6
    diff = rh_cost - pf_cost

    # Calculate Load shedding cost for RH model
    if "Load shedding" in rh_net.generators.index:
        ls_dispatch_rh = rh_net.generators_t.p["Load shedding"]
        ls_cost_rh = rh_net.generators.at["Load shedding", "marginal_cost"]
        ls_total_cost_rh = (ls_dispatch_rh * ls_cost_rh).sum() / 1e6
    else:
        ls_total_cost_rh = 0.0

    # Calculate Load shedding cost for PF model
    if "Load shedding" in pf_net.generators.index:
        ls_dispatch_pf = pf_net.generators_t.p["Load shedding"]
        ls_cost_pf = pf_net.generators.at["Load shedding", "marginal_cost"]
        ls_total_cost_pf = (ls_dispatch_pf * ls_cost_pf).sum() / 1e6
    else:
        ls_total_cost_pf = 0.0

    results.append({
        'year': name_pf.split('_')[2],
        'PF_cost_MEUR': round(pf_cost, 1),
        'RH_cost_MEUR': round(rh_cost, 1),
        'Difference_MEUR': round(diff, 1),
        'PF_Load shedding_cost_MEUR': round(ls_total_cost_pf, 1),
        'RH_Load shedding_cost_MEUR': round(ls_total_cost_rh, 1)
    })

results_df = pd.DataFrame(results)
results_df


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


,year,PF_cost_MEUR,RH_cost_MEUR,Difference_MEUR,PF_Load shedding_cost_MEUR,RH_Load shedding_cost_MEUR
0,1979,11626.3,16507.4,4881.1,0.0,238.8
1,1980,12334.7,13235.4,900.8,0.0,119.4
2,1981,11877.0,15737.5,3860.5,0.0,103.5
3,1982,12393.7,14800.3,2406.6,0.0,263.2
4,1983,12631.1,15650.7,3019.6,0.0,93.8
5,1984,11943.6,11208.7,-734.9,0.0,5.6
6,1985,12324.6,12351.1,26.5,0.0,40.8
7,1986,12204.1,12583.9,379.8,0.0,25.4
8,1987,12386.3,12600.6,214.3,0.0,54.0
9,1988,12180.2,14182.6,2002.4,0.0,127.8


## Functions to calc results

In [ ]:
# FOR expansion model: 

def calculate_cost_recovery_generators(network):
    results = []

    for gen in network.generators.index:
        carrier = network.generators.at[gen, "carrier"]
        bus = network.generators.at[gen, "bus"]
        p_nom = network.generators.at[gen, "p_nom_opt"]
        
        marginal_cost = network.generators.at[gen, "marginal_cost"]
        
        capital_cost = network.generators.at[gen, "capital_cost"]
        
        dispatch = network.generators_t.p[gen]
        prices = network.buses_t.marginal_price[bus]

        # Revenue: price * generation
        revenue = (dispatch * prices).sum()

        # Production cost: generation * marginal cost
        production_cost = (dispatch * marginal_cost).sum()

        # Capital cost: installed capacity * capital cost
        capex = p_nom * capital_cost

        # Profit = revenue - (capex + production cost)
        profit = revenue - (capex + production_cost)

        results.append({
            "name": gen,
            "carrier": carrier,
            "revenue [MEUR]": round(revenue / 1e6, 1),
            "production cost [MEUR]": round(production_cost / 1e6, 1),
            "capital cost [MEUR]": round(capex / 1e6, 1),
            "total cost [MEUR]": round((production_cost + capex) / 1e6, 1),
            "profit [MEUR]": round(profit / 1e6, 1)
        })

    df_results = pd.DataFrame(results)
    return df_results
CR_N_exp = calculate_cost_recovery_generators(N)
CR_N_exp

## Look at results

# Plot results

## Result type 1